In [1]:
# import statements
import numpy as np
import dicom
import cv2 
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy.misc as misc
import sys
import os
sys.path.append("..")
from AlgoEngine.utils import getMeanTargetDose, getImageBlock, getROINumber, getContours

## Inputs to Function

In [2]:
base = '/home/radiation/RadiationTherapyDecisionSupport/data/'
StudyID = 'UCLA_PR_5'
structureset = dicom.read_file(base + StudyID + '/structureset.dcm')

In [3]:
dose_data = dicom.read_file(base + StudyID + "/dose.dcm")
dose_grid = np.array(dose_data.pixel_array)

x_spacing = np.array(dose_data.PixelSpacing[0]).astype(np.uint8)
y_spacing = np.array(dose_data.PixelSpacing[1]).astype(np.uint8)

In [4]:
dose_z = float(dose_data.ImagePositionPatient[2])
ctFilenames = [fl for fl in os.listdir(base + StudyID) if 'CT.' in fl]
ct_sample = dicom.read_file(base + StudyID + '/' + ctFilenames[0])
block_shape = (ct_sample.Rows, ct_sample.Columns, len(ctFilenames))

ct_coords = None

slice_position_z = np.zeros((len(ctFilenames))).astype(np.float32) # Input to Function

for i, fl in enumerate(ctFilenames):
    slice_position_z[i] = dicom.read_file(base + StudyID + '/' + fl).ImagePositionPatient[2]
    ct_struct = dicom.read_file(base + StudyID + '/' + fl)
    if(float(ct_struct.ImagePositionPatient[2]) == dose_z):
        ct_coords = ct_struct.ImagePositionPatient
        ct_spacing = ct_struct.PixelSpacing
                            
slice_position_z = np.sort(slice_position_z)[::-1]

In [5]:
x0 = float(dose_data.ImagePositionPatient[0])- float(ct_coords[0])
x0 = int(round(x0 / float(ct_spacing[0])))
y0 = float(dose_data.ImagePositionPatient[1])- float(ct_coords[1])
y0 = int(round(y0 / float(ct_spacing[1])))

DoseGridScaling = dose_data.DoseGridScaling
imageBlock, sopUID = getImageBlock(StudyID, base)

In [6]:
ROI_NUM = getROINumber(structureset, 'PTV')

roiNumPlanes = len(structureset.ROIContourSequence[ROI_NUM].ContourSequence) 

contour_data = {} # Input to function
image_orientation = {} # Input to function
image_position = {} # Input to function
pixel_spacing = {} # Input to function

for index in range(0, roiNumPlanes):
    
    imageSOP = structureset.ROIContourSequence[ROI_NUM].ContourSequence[index].ContourImageSequence[0].ReferencedSOPInstanceUID
    
    planeContourData = np.array(structureset.ROIContourSequence[ROI_NUM].ContourSequence[index].ContourData)
    planeContourData = planeContourData.reshape(planeContourData.shape[0] // 3 , 3)
    
    contour_data[imageSOP] = planeContourData
    imagei = dicom.read_file(base + StudyID + '/CT.' + imageSOP + '.dcm')
    
    image_orientation[imageSOP] = imagei.ImageOrientationPatient
    image_position[imageSOP] = imagei.ImagePositionPatient
    pixel_spacing[imageSOP] = imagei.PixelSpacing

In [7]:
_, ptv_roi_block = getContours(block_shape, slice_position_z, contour_data, image_orientation, image_position, pixel_spacing)

## Function

In [8]:
dose_mean = getMeanTargetDose(ptv_roi_block, block_shape, dose_grid, DoseGridScaling, x0, y0, x_spacing, y_spacing, sopUID)

## Testing Function

We display the dose mean as we lack a way to validate the code

In [9]:
dose_mean

0.56002593